In [1]:
import argparse 
import datetime
import random

import numpy as np
import tensorflow as tf


args = argparse.Namespace()
args.seed = 101
    
def random_date_output():
    ordinal_min = datetime.date(1000, 1, 1).toordinal()
    ordinal_max = datetime.date(9999, 12, 31).toordinal()
    return datetime.date.fromordinal(random.randint(ordinal_min, ordinal_max))

def output_date_to_input(date) -> str:
    months = ["January", "February", "March", "April", "May", "June",
          "July", "August", "September", "October", "November", "December"]
    
    return "{} {}, {}".format(months[date.month - 1], date.day, date.year)


max_int = ord("z")
def date_to_np(str_date):
    str_date = str_date.lower()
    return np.array([ord(s) for s in str_date])

def np_to_date(np_date):
    return "".join([chr(i) for i in np_date.tolist()])

def build_dataset(dates_num):
    dataset = []
    for i in range(dates_num):
        rd = random_date_output()
        ird = output_date_to_input(rd)
        ird = date_to_np(ird)
        rd = date_to_np(str(rd))
        dataset.append([ird, rd])
    print(dataset)
    dataset = np.array(dataset, dtype=object)
    
    return dataset[:, 0], dataset[:, 1]


train_dataset = build_dataset(2)
np.vstack(train_dataset[1])

[[array([102, 101,  98, 114, 117,  97, 114, 121,  32,  49,  49,  44,  32,
        53,  56,  57,  49]), array([53, 56, 57, 49, 45, 48, 50, 45, 49, 49])], [array([115, 101, 112, 116, 101, 109,  98, 101, 114,  32,  50,  54,  44,
        32,  57,  57,  52,  55]), array([57, 57, 52, 55, 45, 48, 57, 45, 50, 54])]]


array([[53, 56, 57, 49, 45, 48, 50, 45, 49, 49],
       [57, 57, 52, 55, 45, 48, 57, 45, 50, 54]])

In [2]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(max_int, 32, input_shape=[None]))
model.add(tf.keras.layers.LSTM(64))

model(train_dataset[0][0])

ValueError: Exception encountered when calling layer "sequential" (type Sequential).

Input 0 of layer "lstm" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (17, 32)

Call arguments received:
  • inputs=tf.Tensor(shape=(17,), dtype=int64)
  • training=None
  • mask=None

In [56]:
for s in "abc0123, ":
    print(ord(s))
    
[[1, 2],[3, 4]][0][:]
np.array([np.array([1, 2]), np.array([3, 4])])


97
98
99
48
49
50
51
44
32


array([[1, 2],
       [3, 4]])